# Champ_Fonc_MED_Table

In [ ]:
from trustutils import run
import sys, os
run.introduction("Yannick Gorsse")
run.TRUST_parameters()

In [ ]:
run.useMEDCoupling()
import medcoupling as mc
sys.path.append(os.environ["TRUST_ROOT"] + "/Validation/Rapports_automatiques/Verification/PolyMAC/share/maquette_decapoly/src")
from weird_mesh import getWeirdMesh

def build_power_field(n, folder):
    mm = getWeirdMesh(0, 1, 0, 1, list(range(n + 1)), list(range(n + 1)), "o", unpo=True)
    mm.write(f"{folder}/mesh.med", 2)
    m = mm.getMeshAtLevel(0)
    bc = m.computeIsoBarycenterOfNodesPerCell()
    p = m.getMeasureField(True)
    p_a = p.getArray()
    p_a.applyLin(0, 0)
    for i, b in enumerate(bc):
        if b[0] > 0.5: p_a[i] = 1e8
    p.setArray(p_a)
    p.setName("power")
    mc.WriteField(f"{folder}/power.med", p, True)


In [ ]:
from trustutils.jupyter import run

med_syntax = "use_existing_domain file ../power.med domain mesh field power loc elem time 0"
ch_ch = {"champ_fonc_txyz" : ("champ_fonc_txyz mesh 1 1e8*(x>0.5)*(t+1)", "+"),
              "champ_fonc_med_table" : (f"champ_fonc_med_table_temps {{ table_temps {{ 2 0 10 1 11 }} {med_syntax} }}", "x"),
              "champ_fonc_med_table_lue" : (f"champ_fonc_med_table_temps {{ table_temps_lue ../power_evolution.txt {med_syntax} }}", "-")
              }

run.reset()
for ch, (chtrust, _) in ch_ch.items():
    run.addCaseFromTemplate(f"jdd.data", ch, {"power" : chtrust})

with open(f"{run.BUILD_DIRECTORY}/power_evolution.txt", "w") as f:
    f.write("0 1\n10 11")

n = 30

build_power_field(n, run.BUILD_DIRECTORY)

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph("Time steps")

for ch, (chtrust, m) in ch_ch.items():
    s = ""
    if ch.startswith("champ_fonc_med_table"): s = "_power"
    out = plot.loadText(f"{run.BUILD_DIRECTORY}/{ch}/jdd_pbt_Puissance_Thermique{s}.out",transpose=False)
    a.add(out[:, 0], out[:, 1], label=f"{ch}", marker=m, markersize=12)